# Visualizing 3D Uncertainty of Forest Plots Measured with Error
David Diaz

## Objective
Using a probabilistic programming approach, I will incrementally build a data-generating, or *generative* model that will stochastically simulate the locations and shapes of trees on a forest inventory plot.This model will incorporate prior knowledge of common tree forms and expected levels of measurement error in field observations.

Each major step in composing the data generation model will be accompanied with an interactive visualization allowing the viewer to adjust the parameters of each stage in the model see how it affects a 3D representation of a tree or a forest inventory plot.

**The model-building process will culminate with a 3D scene in which the surfaces of trees on a forest inventory plot are animated by sampling the probability distributions of all the parameters that go into the data-generating model.**

## Plot Layout for the Forest Inventory & Analysis (FIA) Program
<img src='./images/Measurements_Plot500.jpg' width='400' align='left'>

<img src='./images/nasa_terrain.jpg' width='400' align='right'>

## 0. Terrain where plot is installed


Using a lidar-derived bare-earth model, we will visualize a 3-D mesh or contour map of the terrain.

#### Interactions
No widgets. The user can rotate and zoom a 3D model of the terrain.



## 1. Location of subplot 1 on the terrain
<img src='./images/step01.jpg' width='400' align='right'>
The center of FIA subplot 1 is calculated from GPS measurements in the field, projected to a UTM coordinate system with X,Y location calculated and assumed to include normally-distributed measurement error in both X and Y directions.

$ X_{plot}[1] \sim N(obs_x, \sigma_{GPS}) $

$ Y_{plot}[1] \sim N(obs_y, \sigma_{GPS}) $

#### Interactions
The user can adjust the X or Y location of the subplot on the terrain.

In [ ]:
## TO DO
# calculate plot location with random error

## 2. Record the location of a tree in subplot 1
Within each FIA subplot, the coordinates of each tree are calculated from the center of that subplot. The location of each tree, $j$, is recorded in the field using an azimuth ($az_{tree,obs}[j]$) and distance ($d_{tree,obs}[j]$) from the subplot center. 

We will convert from the field-recorded azimuth (bearing in degrees from North) to radians and re-orient to the Cartesian coordinate system: 

$\theta_{tree,obs} = \frac{(90- az_{tree}[j])}{2\pi}$

#### Calculations
The (X,Y) coordinates of the stem locations in a subplot are made as:

$X_{tree}[j] = X_{plot}[i] + d_{tree}[j] \cdot cos(\theta_{tree}[j])$

$Y_{tree}[j] = Y_{plot}[i] + d_{tree}[j] \cdot sin(\theta_{tree}[j])$

The elevation of the tree stem at ground level will be calculated based on the elevation of a lidar-derived DEM at that (X,Y) location.

$Z_{tree}[j] = DEM[X_{tree}[j], Y_{tree}[j]]$


<img src='./images/step02.jpg' width='400' align='right'> 
#### Interactions
Widgets allow user to adjust the bearing and distance of a tree from the center of the subplot.

#### Prior Distributions
Both the bearing and the distance of a tree from subplot center are assumed to have normally-distributed measurement error: 

$\theta_{tree}[j] \sim N(\theta_{tree,obs}[j],\sigma_{compass})$

$d_{tree}[j] \sim N(d_{tree,obs}[j], \sigma_{dist})$ 

The re-use of $\sigma_{dist}$ assumes the error in measuring distance between plots is the same as the error in measuring distance to trees from a subplot center.

In [1]:
# convert to cartesian coordinates and convert field-recorded azimuth to radians
theta_tree[j] = np.deg2rad(90 - az_tree[j])

NameError: name 'np' is not defined

## 3. Record the diameter, height, and lean of the tree
The location of a tree top is calculated using the field-measured height, which may be offset in the X,Y directions from the location of the stem based on tree lean. The Z value of the tree top (height above ground) is * **not** * calculated above the DEM at the location of the * **tree top's** * X,Y coordinates. Rather, the Z value of a tree top (top height) is calculated based on observed tree height (with error) plus the elevation of the DEM at the X,Y location of the * **tree stem** *. Envision a right triangle with the height as the vertical side of triangle, and the hypotenuse is a particular angle (lean) from the vertical height measurement.

#### Calculations
The X, Y, Z coordinates of each tree top can be calculated as:

$X_{treetop}[j] = X_{tree}[j] + tan(\phi_{lean}[j]) \cdot height_{tree}[j] \cdot cos(\theta_{lean}[j])$

$Y_{treetop}[j] = Y_{tree}[j] + tan(\phi_{lean}[j]) \cdot height_{tree}[j] \cdot sin(\theta_{lean}[j])$

$Z_{treetop}[j] = Z_{tree}[j] + height_{tree}[j]$

<img src='./images/step03.jpg' width='400' align='right'>
#### Interactions
Widgets allow the user to adjust the diameter, height, angle of lean, and direction of lean. The main stem will be visualized as a cone extending from it's location measured on the ground up to the location of the tree top.

The horizontal offset of the treetop from the base of the tree stem is calculated and decomposed into X and Y components as:

$offset = tan(\phi_{lean}[j]) \cdot height_{tree}[j]$

$offset_X = offset \cdot cos(\theta_{lean}[j])$

$offset_Y = offset \cdot sin(\theta_{lean}[j])$


#### Prior Distributions
Field-measured height ($height_{tree,obs}[j]$) will be modeled with normally-distributed measurement error. The direction of tree lean ($\phi_{lean}[j]$) is uniformly distributed across all possible bearings, i.e., equally as likely to point in any compass direction. The severity of lean ($\theta_{lean}[j]$), which reflects the divergence of the main tree stem from vertical, will be modeled using an exponential distribution (e.g., 90% of trees lean <10 degrees from vertical):

$height_{tree}[j] \sim N(height_{tree,obs}[j], \sigma_{height})$

$\phi_{lean}[j] \sim U[0,2\pi)$

$\theta_{lean}[j] \sim Exp(\lambda)$


## 4. Model a symmetric crown surface
A flexible geometric form, a generalized ellipsoid is used to approximate the three-dimensional crown surface of each tree. A two-dimensional ellipsoid for modeling crowns was originally proposed by Horn (1971): 

<img src='./images/Sheng et al 2001_Figure1.PNG' width='500' align='right'>
$\frac{x^E}{a^E}+\frac{y^E}{b^E}=1$

A three-dimensional variant has been attributed to Pollock (1996), and similar model was also described by Koop (1989).
This model is illustrated by Sheng et al. (2001), who referred to this as the Pollock model.


<img src='./images/step04.jpg' width='400' align='right'>

#### Interactions
Widgets allow user to adjust height to crown base, maximum crown radius, and crown curvature.

In [ ]:
## TO DO
# calculate subplot locations with random error

## 5. Incorporate asymmetry into the crown model

Cescatti (1997) elaborates on the model originally proposed by Horn (1971) and developed by Koop (1989), describing an asymmetric hull crown model in three dimensions, based on 4 crown radii measured in two orthogonal directions, the location of the tree top, and the location of the stem at the bottom of the live crown. This results in a set of eight profiles, four for the upper crown from each measured crown radius on the 'peripheral line' to the tree top, and four for the lower crown from each crown radius on the 'peripheral line' to the stem at the bottom of the live crown.

<img src='./images/Cescatti 1997_Figure1.PNG' width='400' align='left'><img src='./images/Cescatti 1997_Figure3.PNG' width='250' align='left'>

#### Interactions
Widgets allow user to adjust one of the crown profile lines, including maximum crown width, height to maximum crown width, and curvature above and below maximum crown width.




### Allowing neighborhood competition to influence crown asymmetry
Following Seidel et al. (2011), a modifier of crown symmetry may be incorporated based on the competitive influence of neighboring trees. A competition vector is formed for each tree, where the magnitude of the competition vector for each neighboring tree is its diameter at breast height divided by the square root of the distance between the two trees. The sum of these vectors represents the sum of all competitive pressure of the neighbors on the focal tree.

## 6. Add the remaining trees to subplot 1
<img src='./images/step06.jpg' width='400' align='right'>
Additional trees recorded on subplot 1 are automatically added to an interactive 3D visualization.

#### Interactions
No widgets to adjust tree and plot parameters. User can rotate, zoom.

## 7. Measure bearing and distance to center of subplot 2.
The centers of FIA subplots 2, 3, and 4 are calculated using a fixed bearing or azimuth, $\theta_{plot}[i]$, and a distance, $d_{plot}[i]$, of 120 ft from the center of subplot 1. Both the bearing/azimuth and the distance from the center of subplot 1 are assumed to have normally-distributed measurement error.

#### Calculations
<img src='./images/step07.jpg' width='400' align='right'>
For each additional subplot, $i$, where $i \in \{2, 3, 4\}$... the locations of subplots 2, 3, and 4 can be calculated as:

$X_{plot}[i] = X_{plot}[1] + d_{plot}[i]\cdot cos(\theta_{plot}[i])$

$Y_{plot}[i] = Y_{plot}[1] + d_{plot}[i]\cdot sin(\theta_{plot}[i])$

#### Interactions
Widgets enable interactive control of bearing and distance to subplot 2.

#### Prior Distributions

The distance of each plot from subplot 1, as well as the bearings of these subplots from subplot are modeled as: 

$d_{plot}[i] \sim N(120, \sigma_{dist})$

$\theta_{plot}[2] \sim N(\frac{\pi}{2}, \sigma_{compass})$

$\theta_{plot}[3] \sim N(\frac{11\pi}{6},\sigma_{compass})$

$\theta_{plot}[4] \sim N(\frac{7\pi}{6}, \sigma_{compass})$


## 8. Add trees to subplot 2
<img src='./images/step08.jpg' width='400' align='right'> 
The trees observed on subplot 2 are added to the 3D scene.
#### Interactions
No widgets. User can rotate and zoom the 3D visualization of subplots 1 and 2 with trees.

## 9. Add subplots 3 and 4, including trees.
<img src='./images/step09.jpg' width='400' align='right'>
The location of subplots 3 and 4, including trees observed in them will be visualized in 3D.

#### Interactions
No widgets. User can rotate and zoom 3D visualization of all subplots and trees.

## 10. Visualize all subplots and trees, with error.
<img src='./images/step10.jpg' width='400' align='right'>
Animated view of 3D scene, sampling through the probability distributions of the tree and subplot parameters.
